In [17]:
#Script Python no Google Colab. Copiar o script do GitHub e coloar no Colab.
#Link para o Google Colab: https://colab.research.google.com/
#Para utilizar o script faça o upload do arquivo a ser modificado.
#Na linha 21, informe o caminho do arquivo a ser lido:
#Se estiver rodando o script no Google Colab, clicar com o botão direito no arquivo,
#copiar o path e colar substituindo o path no scrip.
#Dúvidas, entrar em contato com Marcelo Brum 21 98057-8932

import pandas as pd
import re

# Função para converter string de tempo para horas decimais
def time_to_decimal(time_str):
    if pd.isnull(time_str):  # Check if the string is NaN or None
        return 0
    factors = {'d': 24, 'h': 1, 'm': 1/60, 's': 1/3600}
    matches = re.findall(r'(\d+)([dhms])', time_str)
    return sum(int(value) * factors[unit] for value, unit in matches)

# Carrega o arquivo Excel, ajustando a linha do cabeçalho caso necessário
df = pd.read_excel('entrada.xlsx', sheet_name='Sheet1', header=1)  # Ajuste o valor do header conforme necessário

# Verifica se a coluna "Duration" está presente
if 'Duration' not in df.columns:
    raise KeyError('A coluna "Duration" não foi encontrada no DataFrame.')

# Aplica a conversão à coluna "Duration"
df['Duration Decimal'] = df['Duration'].apply(time_to_decimal)

# Salva o DataFrame para um novo arquivo Excel
df.to_excel('saida.xlsx', index=False)